In [1]:
import pandas as pd
import numpy as np


In [2]:
df=pd.read_csv('SMHS_general-1.csv')

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

# Separate features and target
X = df.drop(columns=['metabolic_pressure', 'latent_cluster'])
y = df['metabolic_pressure']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  #this ends the data prepartion bit

# Define preprocessing for numeric and categorical columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

# Preprocessing pipelines for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Impute missing values with median
    ('scaler', StandardScaler())  # Scale features to have mean 0, variance 1
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with most frequent category
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode categorical features
])

# Bundle everything into a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [4]:
linear_regression=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model', LinearRegression())
])
linear_regression.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['id', 'age', 'deprivation_index', 'alcohol_units_week',
       'physical_activity_minutes', 'diet_quality_score', 'depression_score',
       'bmi', 'waist_hip_ratio', 'sbp', 'dbp', 'sleep_duration_hours',
       's...
       'fasting_glucose', 'fasting_insulin', 'triglycerides', 'hdl', 'ldl',
       'crp', 'alt', 'family_history_diabetes'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['sex', 'ethnicity', 'smoking_status', 'shift_worker'], dtype='object'))])),
                ('model', LinearRegression())])

In [5]:
y_prediction_linear=linear_regression.predict(X_test)
MSE_linear=mean_squared_error(y_test, y_prediction_linear)
RMSE_linear = np.sqrt(MSE_linear)
r2_linear=r2_score(y_test,y_prediction_linear)

In [6]:
from sklearn.linear_model import Ridge, RidgeCV, Lasso
lasso_regression=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model', Lasso(alpha=0.1))
])
lasso_regression.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['id', 'age', 'deprivation_index', 'alcohol_units_week',
       'physical_activity_minutes', 'diet_quality_score', 'depression_score',
       'bmi', 'waist_hip_ratio', 'sbp', 'dbp', 'sleep_duration_hours',
       's...
       'fasting_glucose', 'fasting_insulin', 'triglycerides', 'hdl', 'ldl',
       'crp', 'alt', 'family_history_diabetes'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['sex', 'ethnicity', 'smoking_status', 'shift_worker'], dtype='object'))])),
                ('model', Lasso(alpha=0.1))])

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

# Define the grid of alpha values to search through
param_grid = {'model__alpha': [0.01, 0.1, 0.5, 1.0, 5.0, 10.0]}

# Set up the pipeline with Lasso and preprocessing
lasso_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', Lasso())
])

# Perform grid search with 5-fold cross-validation
grid_search = GridSearchCV(lasso_pipeline, param_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best model and its alpha value
best_lasso = grid_search.best_estimator_
best_alpha = grid_search.best_params_['model__alpha']

# Predict and evaluate the best model
y_pred_best_lasso = best_lasso.predict(X_test)

rmse_best_lasso = np.sqrt(mean_squared_error(y_test, y_pred_best_lasso))
r2_best_lasso = r2_score(y_test, y_pred_best_lasso)

print(f"Best Lasso Regression RMSE: {rmse_best_lasso}")
print(f"Best Lasso Regression R^2: {r2_best_lasso}")
print(f"Best Alpha Value: {best_alpha}")


Best Lasso Regression RMSE: 5.510860638089254
Best Lasso Regression R^2: 0.8754111079772476
Best Alpha Value: 0.1


In [8]:
y_prediction_lasso=lasso_regression.predict(X_test)
MSE_lasso= mean_squared_error(y_test, y_prediction_lasso)
RMSE_lasso= np.sqrt(MSE_lasso)
r2_lasso= r2_score(y_test, y_prediction_lasso)

In [9]:
from sklearn.tree import DecisionTreeRegressor
DecisionTree=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model', DecisionTreeRegressor(random_state=42))
])
DecisionTree.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['id', 'age', 'deprivation_index', 'alcohol_units_week',
       'physical_activity_minutes', 'diet_quality_score', 'depression_score',
       'bmi', 'waist_hip_ratio', 'sbp', 'dbp', 'sleep_duration_hours',
       's...
       'fasting_glucose', 'fasting_insulin', 'triglycerides', 'hdl', 'ldl',
       'crp', 'alt', 'family_history_diabetes'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['sex', 'ethnicity', 'smoking_status', 'shift_worker'], dtype='object'))])),
                ('model', DecisionTreeRegressor(random_state=42))])

In [10]:
y_prediction_DT=DecisionTree.predict(X_test)
MSE_DT= mean_squared_error(y_test, y_prediction_DT)
RMSE_DT= np.sqrt(MSE_DT)
r2_DT= r2_score(y_test, y_prediction_DT)

In [11]:
comparison= pd.DataFrame({
    'Model':['Linear Regression', 'Lasso Regression', 'Decision Tree'],
    'RMSE': [RMSE_linear, RMSE_lasso,RMSE_DT],
    'R2': [r2_linear, r2_lasso, r2_DT]
})
print(comparison)

               Model      RMSE        R2
0  Linear Regression  5.512715  0.875327
1   Lasso Regression  5.510861  0.875411
2      Decision Tree  7.639306  0.760587
